In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Setear credenciales y nombre del bucket
import os
from google.cloud import storage
import pandas as pd
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/Colab Notebooks/Fifa World Cup - EDA/keys/service-fifa-lg.json'
files = os.listdir('/content/drive/MyDrive/Colab Notebooks/Fifa World Cup - EDA/data')
bucket_name = 'fifa_project'


In [ ]:
# Listar objetos existentes en el bucket de Cloud Storage
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name)
    return [blob.name for blob in blobs]

In [ ]:
list_objects = list_blobs(bucket_name)

In [ ]:

list_objects

['gold_data/matches_worldcup_gold.csv',
 'gold_data/players_gold.csv',
 'gold_data/worldcups_gold.csv',
 'raw_data/WorldCupMatches.csv',
 'raw_data/WorldCupPlayers.csv',
 'raw_data/WorldCups.csv']

In [ ]:
# Leer data de Cloud Storage
worldcups = pd.read_csv( f'gs://{bucket_name}/raw_data/WorldCups.csv')
players_worldcups = pd.read_csv( f'gs://{bucket_name}/raw_data/WorldCupPlayers.csv')
matches_worldcups = pd.read_csv( f'gs://{bucket_name}/raw_data/WorldCupMatches.csv')

In [ ]:
# Transformación de raw data a gold
def data_transform(valor):
  #Función para reemplazar los . por ningun espacio para luego convertir a int64 toda la tabla
  nuevo_valor = valor.replace(".","")
  return nuevo_valor

#Dataframe worldcups
if worldcups['Attendance'].dtype != int:
  worldcups['Attendance'] = worldcups['Attendance'].apply(data_transform) #Aplicar función de data_transform
  worldcups['Attendance'] = worldcups['Attendance'].astype('int64') #Pasar a int64 toda la tabla
  worldcups = worldcups.rename(columns=lambda x: x.replace('-', '_').lower()) #Change the name of the columns to avoid problems in BigQuery


#Datrframes players_worldcups
players = players_worldcups.drop_duplicates()
players_worldcups = players.rename(columns=lambda x: x.replace(' ', '_').lower())
players_worldcups = players_worldcups.rename(columns=lambda x: x.replace('-', '_').lower())
#Dataframes matches_worldcups
matches_wc = matches_worldcups.dropna() #Borrar valores NA de la columna dropna
matches_worldcup = matches_wc.drop_duplicates() #Drop duplicates
#Convertir multiples columnas usando un diccionario
# using dictionary to convert specific columns
convert_dict = {
                'Year': int,
                'Home Team Goals': int,
                'Away Team Goals': int,
                'Attendance':int,
                'Half-time Home Goals':int,
                'Half-time Away Goals':int,
                'RoundID':int,
                'MatchID':int
                }


matches_worldcup = matches_worldcup.astype(convert_dict)
matches_worldcup  = matches_worldcup.rename(columns=lambda x: x.replace(' ', '_').lower()) #Change the name of the columns to avoid problems in BigQuery
matches_worldcup  = matches_worldcup.rename(columns=lambda x: x.replace('-', '_').lower()) #Change the name of the columns to avoid problems in BigQuery





In [ ]:

#Carga de gold data a Cloud Storage
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name)
    return [blob.name for blob in blobs]

def upload_blob(bucket_name, data_frame, destination_blob_name):
    """Uploads a DataFrame to the bucket as a CSV file."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    try:
        # Check if the destination blob already exists in the bucket
        if destination_blob_name in list_blobs(bucket_name):
            print(f"File {destination_blob_name} already exists in the bucket. Skipping upload.")
        else:
            blob = bucket.blob(destination_blob_name)
            blob.upload_from_string(data_frame.to_csv(index=False), content_type='text/csv')
            print(f"DataFrame uploaded to {destination_blob_name}.")
    except Exception as e:
        print(f"An error occurred while uploading DataFrame to {destination_blob_name}: {e}")


dataframes = {
    'worldcups': worldcups,
    'players': players,
    'matches_worldcup': matches_worldcup
}

for df_name, df in dataframes.items():
    destination_blob_name = f'gold_data/{df_name}_gold.csv'
    upload_blob(bucket_name, df, destination_blob_name)

File gold_data/worldcups_gold.csv already exists in the bucket. Skipping upload.
File gold_data/players_gold.csv already exists in the bucket. Skipping upload.
File gold_data/matches_worldcup_gold.csv already exists in the bucket. Skipping upload.


In [ ]:
#Carga de gold data a BigQuery
from google.cloud import bigquery
project_id = 'sqlcursolg'  # Replace with your Google Cloud Project ID
dataset_id = 'gold_data_fifa'  # Replace with the desired BigQuery dataset ID

def create_bigquery_dataset_if_not_exists(client, dataset_id):
    """Create a BigQuery dataset if it does not exist."""
    dataset_ref = client.dataset(dataset_id)
    try:
        client.get_dataset(dataset_ref)
        print(f"Dataset '{dataset_id}' already exists.")
    except Exception as e:
        dataset = bigquery.Dataset(dataset_ref)
        dataset = client.create_dataset(dataset)
        print(f"Dataset '{dataset_id}' created.")

def upload_dataframe_to_bigquery(client, dataframe, table_name):
    """Upload a DataFrame to BigQuery as a new table."""
    table_id = f"{project_id}.{dataset_id}.{table_name}"
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True # Automatically detect schema from DataFrame
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job = client.load_table_from_dataframe(dataframe, table_id, job_config=job_config)
    job.result()  # Wait for the job to complete
    print(f"DataFrame uploaded to BigQuery table: {table_id}")

# Assume you have already loaded 'worldcups', 'players', and 'matches_worldcup' DataFrames

# Create a BigQuery client
bigquery_client = bigquery.Client(project=project_id)

# Create a BigQuery dataset if it does not exist
create_bigquery_dataset_if_not_exists(bigquery_client, dataset_id)

# Upload DataFrames to BigQuery tables

upload_dataframe_to_bigquery(bigquery_client, worldcups, 'worldcups_gold')
upload_dataframe_to_bigquery(bigquery_client, players, 'players_gold')
upload_dataframe_to_bigquery(bigquery_client, matches_worldcup, 'matches_worldcup_gold')


Dataset 'gold_data_fifa' already exists.
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_fifa.worldcups_gold
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_fifa.players_gold
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_fifa.matches_worldcup_gold


In [ ]:
# Transformación de gold data a dashboard gold data dataframe worldcups
# 1 Insights -> (Top paises ganadores copas del mundo)
result_winners = worldcups.get(['country','winner']).groupby('winner',as_index=False).count().sort_values(by='country',ascending=False)
result_winners = result_winners.rename(columns={'Country': 'quantity'})
result_winners = result_winners.reset_index(drop=True)
# 2 Insights -> (Top 10 mundiales con más goles)
worldcups_top_goals = worldcups.sort_values(by='goalsscored',ascending=False).head(10)
# 3 Insights -> (Top 5 mundiales con más asistencias)
attendance_top = worldcups.sort_values(by='attendance',ascending=False).head(5)
# 4 Insights ->  (Paises que han sido sede > 1)
sedes_mundiales = worldcups.get(['country','winner']).groupby('country',as_index=False).count().sort_values(by='winner',ascending=False).reset_index(drop=True)
sedes_mundiales = sedes_mundiales.rename(columns={'winner': 'quantity'})
sedes_mundiales = sedes_mundiales.query('quantity > 1')
# 5 Insights -> (Paises que fueron campeones siendo sede)
campeones_sede = worldcups[['year','country','winner']].query('country == winner')
# 6 Insights -> (Goles,Asistencia y Partidos jugados totales)
metricas = worldcups[['goalsscored','attendance','matchesplayed']].sum().reset_index()
metricas.columns = ['metrica','total']
avg_goal = round(metricas.loc[0][1] / metricas.loc[2][1], 2)
metricas.loc[len(metricas.index)] = ['Avg_Goal_Match ',avg_goal]



In [ ]:
# Transformación de gold data a dashboard gold data dataframe players

# 7 Inisghts -> Top 10 goleadores en mundiales
# Crear función para que me cree columna con el número de goles por partido
def cantidad_goles(cadena):
    return cadena.count('G') + cadena.count('P')
df_players_events = players_worldcups.dropna(subset = ['event']) #Eliminar registros sin eventos
df_players_events['goals'] = df_players_events['event'].apply(cantidad_goles) #Crear columna goles donde salgan los goles por partido
top_10_scorers = df_players_events.get(['player_name','team_initials','goals']).groupby(['player_name','team_initials'],as_index=False).sum().sort_values(by='goals',ascending=False).head(10).reset_index(drop=True)
# 8 Insights -> Top 5 jugadores que anotaron más goles en un partido
top_5_playersgoals_match = df_players_events.sort_values(by='goals',ascending=False).head(5).reset_index(drop=True)
# 9 Insights -> Top 10 jugadores con más partidos jugados
# S --> Inicialista
# N ---> Suplente
# Event --> I or IH46 --> Jugo
df_players_matchs = players_worldcups
df_players_matchs['matchs_line_up'] = df_players_matchs['line_up'].map({'S':1 ,'N':0}) #Crear columna donde se mapea si entro de inicialista o no
df_players_matchs['event'] = df_players_matchs['event'].fillna('N/A') # Llenar valores nulos con el string 'N/A'
# Crear función para que me cree columna con 1 si entro de cambio  o con 0 si se quedo en la banca
def cantidad_partidos(cadena):
    return cadena.count('I')
df_players_matchs['matchs_event'] = df_players_matchs['event'].apply(cantidad_partidos)
df_players_matchs['matchs'] = df_players_matchs['matchs_line_up'] + df_players_matchs['matchs_event'] # Crear columna que sume las dos columnas anteriores
top_10_matchesplayers = df_players_matchs.get(['player_name','team_initials','matchs']).groupby(['player_name','team_initials'],as_index=False).sum().sort_values(by='matchs',ascending=False).head(10).reset_index(drop=True)
# 10 Insights -> Top 10 capitanes con más
goles_capitanes = df_players_events.loc[(df_players_events['position'].isin(['GKC','C']))&(df_players_events['goals']==1)]
capitans_goals = goles_capitanes.get(['player_name','team_initials','goals']).groupby(['player_name','team_initials'],as_index=False).sum().sort_values(by='goals',ascending=False).head(10).reset_index(drop=True)
# 11 Insights -> Top 5 capitanes con más partidos
capitans_matchs = df_players_matchs.loc[(df_players_matchs['position'].isin(['GKC','C']))]
capitans_matchs = capitans_matchs.get(['player_name','team_initials','matchs']).groupby(['player_name','team_initials'],as_index=False).sum().sort_values(by='matchs',ascending=False).head(5).reset_index(drop=True)
# 12 Insights -> Top 10 jugadores con más amarillas en mundiales
def cantidad_amarillas(cadena):
  contador = cadena.count('Y')
  if contador >2:
    return contador - 1
  else:
    return contador
df_players_events['yellow_cards'] = df_players_events['event'].apply(cantidad_amarillas)
yellow_cards_top = df_players_events.get(['player_name','team_initials','yellow_cards']).groupby(['player_name','team_initials'],as_index=False).sum().sort_values(by='yellow_cards',ascending=False).head(10).reset_index(drop=True)
# 13 Insights -> Top 10 jugadores con más amarillas en mundiales
def cantidad_rojas(cadena):
    return cadena.count('R')
df_players_events['red_cards'] = df_players_events['event'].apply(cantidad_rojas)
red_cards_top = df_players_events.get(['player_name','team_initials','red_cards']).groupby(['player_name','team_initials'],as_index=False).sum().sort_values(by='red_cards',ascending=False).head(10).reset_index(drop=True)
# 13 Insights -> Top 5 jugadores con más goles entrando de suplentes
players_goal_in = df_players_events.loc[(df_players_events['event'].str.contains(pat='I')) & ((df_players_events['event'].str.contains(pat='G'))|(df_players_events['event'].str.contains(pat='P')))]
players_goal_in = players_goal_in.get(['player_name','team_initials','goals']).groupby(['player_name','team_initials'],as_index=False).sum().sort_values(by='goals',ascending=False).head(5).reset_index(drop=True)
# 14 Insights --> Top 5 jugadores con más autogoles en mundiales
def cantidad_autgoles(cadena):
    return cadena.count('W')
auto_goal = df_players_events.loc[(df_players_events['event'].str.contains(pat='W'))]
auto_goal['auto_goals'] = auto_goal['event'].apply(cantidad_autgoles)
players_auto_goal = auto_goal.get(['player_name','team_initials','auto_goals']).groupby(['player_name','team_initials'],as_index=False).sum().sort_values(by='auto_goals',ascending=False).reset_index(drop=True)



<ipython-input-81-f10fd4643650>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auto_goal['auto_goals'] = auto_goal['event'].apply(cantidad_autgoles)


In [ ]:
# Transformación de gold data a dashboard gold data dataframe matches
# 15 Insights --> Top 10 Arbitros con más partidos
referee_counts = matches_worldcup['referee'].value_counts()
assistant1_counts = matches_worldcup['assistant_1'].value_counts()
assistant2_counts = matches_worldcup['assistant_2'].value_counts()
combined_counts = pd.concat([referee_counts, assistant1_counts, assistant2_counts], axis=1, sort=True)
combined_counts.columns = ['referee', 'assistant_1', 'assistant_2']
combined_counts = combined_counts.fillna(0)
combined_counts['total_appearances'] = combined_counts['referee'] + combined_counts['assistant_1'] + combined_counts['assistant_2']
top_matches_referees = combined_counts.reset_index().rename(columns={'index': 'person'}).sort_values(by='total_appearances', ascending=False).head(10).reset_index(drop=True)
# 16 Insights --> Top 10 partidos con más goles
matches_worldcup['total_goals'] = matches_worldcup['home_team_goals'] + matches_worldcup['away_team_goals']
top_matches_goals = matches_worldcup.get(['year','stage','home_team_name','home_team_goals','away_team_goals','away_team_name','total_goals']).sort_values(by='total_goals',ascending=False).reset_index(drop=True).head(10)
# 16 Insights --> Top 10 goleadas historicas de locales
top_local_goals = matches_worldcup.get(['year','stage','home_team_name','home_team_goals','away_team_goals','away_team_name']).sort_values(by='home_team_goals',ascending=False).reset_index(drop=True).head(10)
# 17 Insights --> Top 10 goleadas historicas de visitantes
top_away_goals = matches_worldcup.get(['year','stage','home_team_name','home_team_goals','away_team_goals','away_team_name']).sort_values(by='away_team_goals',ascending=False).reset_index(drop=True).head(10)
# 18 Insights --> Remontada de visitantes despues de ir perdiendo finalizando el primer tiempo
remontada_visitantes = matches_worldcup.loc[(matches_worldcup['half_time_home_goals'] > matches_worldcup['half_time_away_goals']) & (matches_worldcup['away_team_goals'] > matches_worldcup['home_team_goals'])]
# 19 Insights --> Remontada de locales despues de ir perdiendo finalizando el primer tiempo
remontada_locales = matches_worldcup.loc[(matches_worldcup['half_time_home_goals'] < matches_worldcup['half_time_away_goals']) & (matches_worldcup['away_team_goals'] < matches_worldcup['home_team_goals'])]
# 20 Insights --> Top 10 partidos con más asistencias
top_10_attendance_matches = matches_worldcup.get(['datetime','stage','stadium','city','home_team_name','home_team_goals','away_team_goals','away_team_name','attendance']).sort_values(by='attendance',ascending=False).reset_index(drop=True).head(10)
# 21 Insights --> Top 10 grupo con más goles por mundial
groups_matcheswc = matches_worldcup.loc[(matches_worldcup['stage'].isin(['Group 1', 'Group 4', 'Group 2', 'Group 3','Group 5','Group 6','Group B', 'Group A',
                                                      'Group C', 'Group D', 'Group F', 'Group E','Group H', 'Group G']))]
top_10_goals_group = groups_matcheswc.get(['year','stage','total_goals']).groupby(['year','stage'],as_index=False).sum().sort_values(by='total_goals',ascending=False).head(10).reset_index(drop=True)
# 22 Insights --> Top 10 finales con más asistentes
top_10_finals_attendance = matches_worldcup.get(['datetime','stage','stadium','city','home_team_name','home_team_goals','away_team_goals','away_team_name','win_conditions','attendance','total_goals'])\
.loc[(matches_worldcup['stage'] == 'Final')]\
.sort_values(by='attendance',ascending=False).reset_index(drop=True).head(10)
# 23 Insights --> Top 10 finales con más goles
top_10_finals_goals = matches_worldcup.get(['datetime','stage','stadium','city','home_team_name','home_team_goals','away_team_goals','away_team_name','win_conditions','attendance','total_goals'])\
.loc[(matches_worldcup['stage'] == 'Final')]\
.sort_values(by='total_goals',ascending=False).reset_index(drop=True).head(10)

# 24 Insights --> Finales ganadas en penaltis
finals_wins_penalties = matches_worldcup.get(['datetime','stage','stadium','city','home_team_name','home_team_goals','away_team_goals','away_team_name','win_conditions','attendance','total_goals'])\
.loc[(matches_worldcup['stage'] == 'Final') & (matches_worldcup['win_conditions'].str.contains(pat='penalties'))]

# 25 Insights --> Partido con más amarillas
most_yellows_cards = df_players_events.get(['matchid','yellow_cards']).groupby(['matchid'],as_index=False).sum().sort_values(by='yellow_cards',ascending=False)
most_yellows_cards_matchs =pd.merge(most_yellows_cards,matches_worldcup. \
         get(['datetime','stage','stadium','city','home_team_name','home_team_goals','away_team_goals','away_team_name','matchid']),on ='matchid').head(2)
# 26 Insights --> Partido con más rojas
most_red_cards = df_players_events.get(['matchid','red_cards']).groupby(['matchid'],as_index=False).sum().sort_values(by='red_cards',ascending=False)
most_red_cards_matchs =  pd.merge(most_red_cards,matches_worldcup. \
         get(['datetime','stage','stadium','city','home_team_name','home_team_goals','away_team_goals','away_team_name','matchid']),on ='matchid').head(1)
# 27 Insights --> Mundiales con sus respectivos autogoles
matches_world = matches_worldcup.get(['year','matchid'])
auto_goals_year = pd.merge(matches_world,auto_goal.get(['matchid','auto_goals']),how='inner',on='matchid')
auto_goal_wc = auto_goals_year.get(['year','auto_goals']).groupby(['year'],as_index=False).sum()
auto_goal_worldcup = pd.merge(worldcups.get(['year','country']),auto_goal_wc,how='inner',on='year').sort_values(by='auto_goals',ascending=False).reset_index(drop=True)

In [ ]:

# Carga de gold data dashboard a Cloud Storage
dataframes_dashboards = {
    'result_winners': result_winners,
    'worldcups_top_goals': worldcups_top_goals,
    'attendance_top_wc': attendance_top,
    'sedes_mundiales_top': sedes_mundiales,
    'campeones_sede': campeones_sede,
    'metricas': metricas,
    'top_10_scorers': top_10_scorers,
    'top_5_playersgoals_match':top_5_playersgoals_match,
    'top_10_matchesplayers':top_10_matchesplayers,
    'capitans_goals':capitans_goals,
    'capitans_matchs':capitans_matchs,
    'yellow_cards_top':yellow_cards_top,
    'red_cards_top':red_cards_top,
    'players_goal_in':players_goal_in,
    'players_auto_goal':players_auto_goal,
    'top_matches_referees':top_matches_referees,
    'top_matches_goals':top_matches_goals,
    'top_local_goals':top_local_goals,
    'top_away_goals':top_away_goals,
    'remontada_visitantes':remontada_visitantes,
    'remontada_locales':remontada_locales,
    'top_10_attendance_matches':groups_matcheswc,
    'top_10_goals_group':top_10_goals_group,
    'top_10_finals_attendance':top_10_finals_attendance,
    'top_10_finals_goals':top_10_finals_goals,
    'finals_wins_penalties':finals_wins_penalties,
    'most_yellows_cards_matchs':most_yellows_cards_matchs,
    'most_red_cards_matchs':most_red_cards_matchs,
    'most_red_cards_matchs':auto_goal_worldcup,
}

for df_name, df in dataframes_dashboards.items():
    destination_blob_name = f'gold_data/dashboards/{df_name}_gold.csv'
    upload_blob(bucket_name, df, destination_blob_name)

File gold_data/dashboards/result_winners_gold.csv already exists in the bucket. Skipping upload.
File gold_data/dashboards/worldcups_top_goals_gold.csv already exists in the bucket. Skipping upload.
File gold_data/dashboards/attendance_top_wc_gold.csv already exists in the bucket. Skipping upload.
File gold_data/dashboards/sedes_mundiales_top_gold.csv already exists in the bucket. Skipping upload.
File gold_data/dashboards/campeones_sede_gold.csv already exists in the bucket. Skipping upload.
File gold_data/dashboards/metricas_gold.csv already exists in the bucket. Skipping upload.
File gold_data/dashboards/top_10_scorers_gold.csv already exists in the bucket. Skipping upload.
File gold_data/dashboards/top_5_playersgoals_match_gold.csv already exists in the bucket. Skipping upload.
File gold_data/dashboards/top_10_matchesplayers_gold.csv already exists in the bucket. Skipping upload.
File gold_data/dashboards/capitans_goals_gold.csv already exists in the bucket. Skipping upload.
File g

In [ ]:
#Carga de gold data dashboard a BigQuery
from google.cloud import bigquery
project_id = 'sqlcursolg'  # Replace with your Google Cloud Project ID
dataset_id = 'gold_data_dashboards_fifa'  # Replace with the desired BigQuery dataset ID

# Create a BigQuery client
bigquery_client = bigquery.Client(project=project_id)

# Create a BigQuery dataset if it does not exist
create_bigquery_dataset_if_not_exists(bigquery_client, dataset_id)

for df_name, df in dataframes_dashboards.items():
    table_name = f'{df_name}_dashboard'
    upload_dataframe_to_bigquery(bigquery_client,df,table_name)



Dataset 'gold_data_dashboards_fifa' already exists.
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.result_winners_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.worldcups_top_goals_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.attendance_top_wc_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.sedes_mundiales_top_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.campeones_sede_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.metricas_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.top_10_scorers_dashboard


/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:603: UserWarning: Pyarrow could not determine the type of columns: position.
  warnings.warn(


DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.top_5_playersgoals_match_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.top_10_matchesplayers_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.capitans_goals_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.capitans_matchs_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.yellow_cards_top_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.red_cards_top_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.players_goal_in_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.players_auto_goal_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_fifa.top_matches_referees_dashboard
DataFrame uploaded to BigQuery table: sqlcursolg.gold_data_dashboards_f